In [1]:
#python packages pd
import numpy as np
import matplotlib.pyplot as plt

#machine learning packages
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional
from keras.layers import CuDNNLSTM
from keras.utils.np_utils import to_categorical
# from keras.callbacks import EarlyStopping
from keras.layers import Dropout

from sklearn.model_selection import train_test_split

import importlib

#custom python scripts
import generator

import utilis

Using TensorFlow backend.


In [ ]:
# Check that you are running GPU's
utilis.GPU_checker()

In [3]:
utilis.aws_setup()

AWS SETUP SHOULD BE COMPLETE, we are on <botocore.client.S3 object at 0x7ff8085ac400>


In [4]:
%%time
#  generators
importlib.reload(generator)
training_generator = generator.Keras_DataGenerator( dataset='train', w_hyp=False)
validation_generator = generator.Keras_DataGenerator(dataset='valid', w_hyp= False)

Retrieving data from deephol-data-processed/proofs/human/train/
Generating examples from a set of 376968 examples
Retrieving data from deephol-data-processed/proofs/human/valid/
Generating examples from a set of 104054 examples
CPU times: user 2.43 s, sys: 1.22 s, total: 3.65 s
Wall time: 13 s


In [5]:
#Constants
# ARE YOU LOADING MODE?
VOCAB_SIZE = 1254
INPUT_LENGTH = 3000
EMBEDDING_DIM = 256

# model
def build_model(vocab_size, embedding_dim, input_length):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=input_length))
    model.add(SpatialDropout1D(0.4))
    model.add(Bidirectional(CuDNNLSTM(128)))
    model.add(Dropout(0.4))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(41, activation='softmax'))
    return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, INPUT_LENGTH)

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 3000, 128)         160512    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 3000, 128)         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               264192    
_________________________________________________________________
dense_1 (Dense)              (None, 41)                10537     
Total params: 435,241
Trainable params: 435,241
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
## WARNING IF YOU CONTAIN MULTIPLE CORE GPUS
# NOTE unclear if these causes a speed up 
# @TANCREDI, I HAVE TREID THIS ON JUST GOALS AND DOES NOT SEEM TO CAUSE A SPEED UP MAY 
#CAUSE A SPEED UP IF WE USE HYPOTHESIS
from keras.utils import multi_gpu_model
model_GPU = multi_gpu_model(model, gpus= 4)
model_GPU.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
## ARE YOU LOADING A MODEL IF YES RUN THE FOLLOWING LINES 
# from keras.models import model_from_json
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("model.h5")
# print("Loaded model from disk")
# # REMEMEBER TO COMPILE 
# loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
#overwriting model
# model = loaded_model

In [ ]:
# from keras.utils import multi_gpu_model
# model_GPU = multi_gpu_model(model, gpus= 4)
# model_GPU.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
%%time
n_epochs = 1
history = model_GPU.fit_generator(generator=training_generator,
#                         validation_data=validation_generator,
                        verbose=1,
                        use_multiprocessing= False,
                        epochs=n_epochs)

Epoch 1/1
5814/5814 [==============================] - 3150s 542ms/step - loss: 2.7097 - acc: 0.1644
CPU times: user 2h 8min 28s, sys: 7min 25s, total: 2h 15min 54s
Wall time: 52min 36s


In [22]:
# FOR SAVING MODEL
model_json = model_GPU.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_GPU.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [23]:
#WARNING_DECIDE_HOW_TO_NAME_LOG
#descriptionofmodel_personwhostartsrun
#e.g. LSTM_128encoder_etc_tanc
LOSS_FILE_NAME = "BiLSTM_goals_hyp_large_manu_ep1"

#WARNING NUMBER 2 - CURRENTLY EVERYTIME YOU RERUN THE CELLS BELOW THE FILES WITH THOSE NAMES GET WRITTEN OVER

In [24]:
# save history - WARNING FILE NAME
utilis.history_saver_bad(history, LOSS_FILE_NAME)

SAVED SOME LOGS -- OVERWROTE OLD LOGS -- SOMEONE NEEDS TO FIX THIS


In [ ]:
# read numpy array
# history_toplot = np.genfromtxt("training_logs/"+ LOSS_FILE_NAME +".csv")
# plt.plot(history_toplot)
# plt.title('Loss history')
# plt.show()